# Exercise 1.4.2 - Building Custom CNNs
#### By Jonathan L. Moran (jonathan.moran107@gmail.com)
From the Self-Driving Car Engineer Nanodegree programme offered at Udacity.

## Objectives

* Use the TensorFlow Keras [Sequential API](https://www.tensorflow.org/api_docs/python/tf/keras/Sequential) to create a convolutional neural network ([CNN](https://en.wikipedia.org/wiki/Convolutional_neural_network));
* Implement the [LeNet-5](https://en.wikipedia.org/wiki/LeNet) CNN architecture;
* Train the LeNet-5 on the modified [MNIST digits](http://yann.lecun.com/exdb/mnist/) dataset;
* Train a CNN model (e.g., LeNet-5) on the German Traffic Sign Recognition Benchmark [GTSRB](https://benchmark.ini.rub.de/gtsrb_dataset.html) dataset.

## 1. Introduction

In [ ]:
### Importing the required modules

In [ ]:
import logging
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.preprocessing import image_dataset_from_directory

### 1.1. Convolutional Neural Networks (CNNs)

#### Background

A [convolutional neural network](https://en.wikipedia.org/wiki/Convolutional_neural_network) ("ConvNet") is a special type of neural network particularly well-suited to handle image data. Like the feedforward neural networks we saw in [Exercise 1.3.3](https://github.com/jonathanloganmoran/ND0013-Self-Driving-Car-Engineer/blob/main/1-Object-Detection-in-Urban-Environments/Exercises/1-3-3-Image-Classification-FNNs/2022-09-05-Image-Classification-Feed-Forward-Neural-Networks.ipynb), CNNs are composed of layers of neurons that have trainable weights and biases. In addition, CNNs often use activation functions and fully-connected layers similar to FNNs (e.g., softmax).

With CNNs, we make the assumption that we will always be using image data as our inputs. Since image data is multi-dimensional, resizing these $\left[H, W, D\right]$ arrays to flattened $\left[1x\left(H*W*D\right)\right]$ vectors is necessary when using fully-connected layers, such as in feedforward neural networks. With ConvNets, however, we make use of locally-connected layers that operate over small portions of _volumetric data_. That is, ConvNets allow us to preserve the existing structure of image data and drastically reduce the amount of trainable parameters needed in each layer of our network. For example, a single fully-connected neuron will require a number of weight values equal to the number of attributes in our image input. For an image of size $\left[64, 64, 3\right]$, we obtain $(64*64*3) = 12288$ weight values _per neuron_. For a layer with only $n = 10$ fully-connected neurons, the number of trainable weights we have skyrockets to an astonishing $n * 12288 = 122880$ parameters per layer. We will see in this exercise that by limiting the connectivity of neurons between layers, we can design a network that reduces the number of parameters required while simultaneously _increasing_ the number of layers in the architecture.

![Fig. 1. The convolutional neural network architecture (simplified).](figures/2022-09-12-Figure-1-Convolutional-Neural-Network-Simplified.png)

$$
\textrm{Fig. 1. The convolutional neural network architecture (simplified)}.
$$

#### Layers

Most, if not all, convolutional neural network architectures use some combination of the following three distinct layer types: _convolutional layer_, _pooling layer_, and _fully-connected layer_. We have implemented fully-connected layers in [Exercise 1.3.3](https://github.com/jonathanloganmoran/ND0013-Self-Driving-Car-Engineer/blob/main/1-Object-Detection-in-Urban-Environments/Exercises/1-3-3-Image-Classification-FNNs/2022-09-05-Image-Classification-Feed-Forward-Neural-Networks.ipynb) for feedforward neural networks, and their application in ConvNets is no different. However, we introduce the _convolutional layer_ in this exercise. We will touch on these in more detail in just a minute.

##### Input layer

It is important to first note that our _input layer_ corresponds directly to the dimensionality of our image data. One major assumption we make with convolutional neural networks is that our input image dimensions are constant. That is, that every image passing through our network shares the same _height_, _width_, and _channel_ dimensions. In ConvNets, the _depth_ of an image (its colour channels, e.g., R, G, B) is maintained throughout the network while the height and width might change. For convolutional and pooling layers, this matters because each kernel and filter will require an extra dimension $D$ equal to the number of colour channels in an image in addition to their width and height attributes. For an RGB image, a kernel/filter of size $\left(h \ x \ w\right)$ will therefore have dimensions $\left(h \ x \ w \ x \ 3\right)$.

##### Convolutional layer

Very similar to the pooling layers we studied in [Exercise 1.4.1](https://github.com/jonathanloganmoran/ND0013-Self-Driving-Car-Engineer/blob/main/1-Object-Detection-in-Urban-Environments/Exercises/1-4-1-Pooling-Layers-CNNs/2022-09-07-Pooling-Layers-Convolutional-Neural-Networks.ipynb), these layers take an input, the volumetric image data, and perform sliding window operations ([convolutions](https://en.wikipedia.org/wiki/Convolution)) over the image. The convolution layer, like pooling layers, have _stride_, _kernel size_ and _padding_ attributes. We also introduce a _filter_ hyperparameter which allows us to specify the desired dimensionality of our convolution layer output. Convolutional layers rarely ever have just _one filter_. In fact, it is very common to see a single convolutional layer have anywhere from 32 filters up to 512. Each _filter_ represents a _feature_ in the input image (hence the output name "feature map"). These filters are templates which, when convolved with an input image, help locate and identify vertical, horizontal edges, arches, diagonal lines, etc.

The convolutional layer in a network is often directly followed by an element-wise activation layer such as the Rectified Linear Unit ([ReLU](https://en.wikipedia.org/wiki/Rectifier_\(neural_networks\))) to introduce further non-linearities into the network. Note that the activation layer's output dimensions remain constant.

##### Pooling layer

Pooling layers, as mentioned in the previous Exercise 1.4.1, reduce the size (spatial dimensions) of the images through a downsampling (averaging) or sub-sampling (max/min) operation. By replacing adjacent pixel values with neighbourhood calculations we are able to transform the output and effectively manipulate (increase _multiplicatively_) the [receptive field](https://en.wikipedia.org/wiki/Receptive_field#In_the_context_of_neural_networks) of the image (i.e., the region within an image of which a particular neuron is sensitive to). On the other hand, convolutional layers increase the receptive field _linearly_ proportional to the kernel size. Cascading convolutional layers with _dialated filters_ increase the receptive field exponentially [1]. By shrinking the image, we are essentially _increasing_ the amount of its pixels (information) we can fit into a single "patch" for a neuron to consider.

As the image becomes reduced in size as it passes through the network, neurons in deeper layers get to "see" larger and larger parts of the whole image. In turn, we can increase the receptive field of a network exponentially while increasing the number of parameters linearly. This trade-off is what affords us the ability to swap only a few fully-connected layers for _handfuls_ of convolutional stacks without increasing complexity or decreasing the receptive field.

##### Fully-connected layer

While we aren't able to eliminate fully-connected layers altogether, their use in convolutional neural networks is limited. Fully-connected layers appear at the end of a network for classification; the number of neurons of the last fully-connected layer in a CNN equal the number of distinct classes predicted (e.g., there are $3$ distinct classes being predicted in the example shown in Fig. 1). Practitioners may also choose to implement several preceding fully-connected layers with an arbitrarily large number of neurons, e.g., 512, 256, 64.

#### In summary
   * Inputs to a ConvNet are volumetric image data with fixed dimensions;
   * We use pooling layers to _downsample_ or _sub-sample_ images (reduce their spatial dimensions);
   * By reducing the spatial dimensions, we _increase_ the neurons' receptive fields (how much of the image they can "see");
   * With convolutional networks we are able to eventually cover the entire region of an input image at a cost significantly reduced to fully-connected layers;

We have now learned the benefits of convolutional architectures to more-traditional neural networks (specifically, feedforward neural networks); by adding "stacks" of convolutional/pooling layers, we can exploit the complexity-efficiency trade-off. Convolutional stacks help us set reasonable limits on neural network complexity by decreasing the number of overall trainable parameters we have to work with layer-for-layer. This not only improves training efficiency but also helps boost performance on image classification tasks.

While popular CNN architectures often make use of even more efficiency-focused layers, such as _dropout_/_inverted dropout_, _skip connections_, and _batch normalisation_, we will limit our discussion on those topics for now.

### 1.2. LeNet

#### Background

![Fig. 2. The LeNet-5 convolutional neural network architecture for digits recognition.](figures/2022-09-12-Figure-2-LeNet-5-CNN-Architecture.png)

$$
\textrm{Fig. 2. The LeNet-5 convolutional neural network architecture for digits recognition (proposed in LeCun 1998:7).}
$$

The [LeNet-5](https://en.wikipedia.org/wiki/LeNet) is a convolutional neural network architecture proposed by Yann LeCun et al. of Bell Labs in 1998 [2]. LeNet-5 is recognised in the deep learning community as being one of the earliest ConvNet architectures with work dating back to [1989](https://doi.org/10.1162%2Fneco.1989.1.4.541). LeCun's model (illustrated in Fig. 2) has all the basic units of a convolutional neural network: convolutional layers, pooling layers and fully-connected layers. Like most ConvNet architectures we studied, the LeNet-5 has alternating convolutional and sub-sampling (average pooling) layers. Also as expected we see that the number of filters/kernel windows increases as we go deeper in the network, starting with $6$ and increasing to $16$ in the final convolutional layer. In Fig. 2, we can interpret the number preceeding the "@" symbol as the number of filters/kernels of each layer, followed by the filter/kernel size (omitting the _depth_ dimension from the notation). Lastly, we have for each layer in the convolutional stack a _stride_ hyperparameter denoted by $s$.

Not shown in Fig. 2 is the use of the [tanh](https://paperswithcode.com/method/tanh-activation) non-linear activation function throughout the network. While this proved to be a better performer than the sigmoid function for a multi-layer neural network, _tanh_ did not appropriately handle the [vanishing gradient problem](https://en.wikipedia.org/wiki/Vanishing_gradient_problem), architectures today utilise other activation functions such as [ReLU](https://en.wikipedia.org/wiki/ReLU) or [batch normalisation](https://en.wikipedia.org/wiki/Batch_normalization) to solve this. The last layer of the network utilises a [softmax](https://en.wikipedia.org/wiki/Softmax_function) activation function to fit a Gaussian distribution to the predicted class probabilities.

#### Layers

##### Input layer

The input to the LeNet-5 network is a set of $\left(32 \ x \ 32 \ x \ 1\right)$ grayscale images, each consisting of a single handwritten digit (read more on the modified MNIST SD-1/SD-3 dataset [here](http://yann.lecun.com/exdb/mnist/)). The pixel values of the input images are normalised (anti-aliasing) so that the background (white) has a pixel value of $-0.1$ and the foreground (black) has a value $1.175$. This choice for normalisation is made such that the mean value of the input is roughly $0$ and the variance roughly $1$ to accelerate learning.

##### Convolutional layer (C1)

* **Input size**: $32 x 32 x 1$
* **Feature maps**: $6$
* **Kernel size**: $5x5x1$
* **Stride**: $1$
* **Padding**: $0$
* **Output size**: $(28x28x1)$; $(32 - 5 + 2*0) \ / \ 1 + 1 = 28$
* **Neurons**: $6 * (28x28x1) = 4704$
* **Trainable parameters**: $6 * (5x5x1 + 1) = 156$
* **Total connections**: $6 * (5x5x1 + 1) * (28x28) = 122304$
* **Activation function**: Tanh

This convolutional layer has $6$ feature maps. Each unit of each feature map connects to a $5 x 5$ region within the input image. The feature map size of $28 x 28$ has been selected so that the strided convolutions do not result in any dropped pixels (for more information on that, see [Exercise 1.4.1](https://github.com/jonathanloganmoran/ND0013-Self-Driving-Car-Engineer/blob/main/1-Object-Detection-in-Urban-Environments/Exercises/1-4-1-Pooling-Layers-CNNs/2022-09-07-Pooling-Layers-Convolutional-Neural-Networks.ipynb)). each kernel has an additional bias term (hence the "+1" in the above calculation of trainable parameters. While the number of connections seems tremendous, due to _weight sharing_ we only need to learn $156$ parameters for this layer.

##### Average pooling layer (S2)

* **Input size**: $28 x 28 x 1$
* **Feature maps**: $6$
* **Filter size**: $2x2x1$
* **Stride**: $2$
* **Padding**: $0$
* **Output size**: $(14x14x1)$; $(28 - 2 + 2*0) \ / \ 2 + 1 = 14$
* **Neurons**: $6 * (14x14x1) = 1176$
* **Trainable parameters**: $6 * (1 + 1) = 12$
* **Total connections**: $6 * (2x2x1 + 1) * (14x14x1) = 5880$
* **Activation function**: Tanh

This sub-sampling layer uses the local average pooling calculations to reduce the input size in half from $28x28$ to $14x14$. Each window of values is summed, multiplied by a weight coefficient $w$ and then added together with a trainable bias offset term $b$ (hence the number of trainable parameters shown above). These parameters are learned with [back-propagation](https://en.wikipedia.org/wiki/Backpropagation). The intermediate output values are then passed into a sigmoid activation function which is used to enforce non-linearity. The coefficient and bias terms regulate the intensity of the sigmoid function on the output. If the coefficient values are small, then the units operate in a quasi-linear mode such that the sub-sampling layer merely blurs the input. If the coefficient is large, sub-sampling performs similarly to a "noisy OR", "noisy AND" function depending on the value of the bias (LeCun 1998).

The affect of downsampling of the input is a reduction in sensitivity of the output to shifts and distortions. By progressively reducing the spatial resolution with sub-sampling, we compensate the effect by progressively increasing the number of feature maps (the richness of the representation).

##### Convolutional layer (C3)

* **Input size**: $14 x 14 x 1$
* **Feature maps**: $16$
* **Kernel size**: $5 x 5 x 1$
* **Stride**: $1$
* **Padding**: $0$
* **Output size**: $(10x10x1)$; $(14 - 5 + 2*0) \ / \ 1 + 1 = 10$
* **Neurons**: $16 * (10x10x1) = 1600$
* **Trainable parameters**: $6 * \left[(3x5x5 + 1) + (4x5x5 + 1)\right] + 3*\left[(4x5x5 + 1)\right] + 1*\left[(6x5x5 + 1)\right] = 1516$
* **Total connections**: $1516*(10x10x1) = 151600$
* **Activation function**: Tanh

In this second convolutional layer, we increase the number of feature maps to $16$. In conjunction with the progressive decrease in spatial dimensions of the input images, we now _increase_ the richness of the representation of the input by increasing the number of feature maps. With the increase in feature maps brings a non-complete connection scheme in order to keep the number of connections of $S2$ and $C3$ within resonable bounds.

![Fig. 3. The non-complete connection scheme for layers S2 and C3 (reproduced from LeCun et al., 1998)](figures/2022-09-12-Figure-3-Table-1-LeCun-1998.png)

$$
\textrm{Fig. 3. The non-complete connection scheme for layers} \ S2  \ \textrm{and} \  C3  \ \textrm{(reproduced from LeCun et al., 1998)}
$$

From the _total connections_ calculation above we see that the first _six_ feature maps of $C3$ take inputs from every contiguous subset of _three_ feature maps of $S2$. The next _six_ feature maps of $C3$ take inputs from every contiguous subset of _four_ feature maps of $S2$. The following _three_ feature maps of $C3$ take inputs from _discontinuous_ subsets of _four_ feature maps of $S2$. The last feature map ($1$) takes _all six_ inputs of the feature maps of $S2$.

With that, we calculate the total number of trainable parameters of $C3$ equal to $1516$ and the number of total connections equal to $151600$.

##### Average Pooling layer (S4)

* **Input size**: $10 x 10 x 1$
* **Feature maps**: $16$
* **Filter size**: $2x2x1$
* **Stride**: $2$
* **Padding**: $0$
* **Output size**: $(5x5x1)$; $(10 - 2 + 2*0) \ / \ 2 + 1 = 5$
* **Neurons**: $16 * (5x5x1) = 400$
* **Trainable parameters**: $16 * (1 + 1) = 32$
* **Total connections**: $16 * (2x2x1 + 1) * (5x5x1) = 2000$
* **Activation function**: Tanh

The down-sampling layer $S4$ reduces its input by a factor of 2 from $10x10x1$ down to $5x5x1$. The $16$ feature maps of size $10x10x1$ from $C3$ are pooled into $2x2x1$ units to obtain $16$ feature maps of size $5x5x1$.  

##### Convolutional layer (C5)

* **Input size**: $5 x 5 x 1$
* **Feature maps**: $120$
* **Kernel size**: $5 x 5 x 1$
* **Stride**: $1$
* **Padding**: $0$
* **Output size**: $(1x1x1)$; $(5 - 5 + 2*0) \ / \ 1 + 1 = 1$
* **Neurons**: $120 * (1x1x1) = 120$
* **Trainable parameters**: $120 * (16x5x5x1 + 1) = 48120$
* **Total connections**: $120 * (16x5x5x1) = 48000$
* **Activation function**: Tanh

Here the final convolutional layer $C5$ is considered to be fully-connected with respect to the input dimensionality of the digits images. For any other configuration (i.e., an increase in the number of channels, as with RGB images), we can expect this to be explicitly a convolutional layer. For the digits dataset, given a kernel size of $5x5x1$ and a corresponding output of layer $S4$ of size $5x5x1$, we obtain a fully-connected layer with $(1x1x1)$ feature map output dimensions. Therefore, we have $48120$ trainable parameters and $48000$ total connections in this last convolutional layer.  

##### Fully-connected layer (F6)

* **Input size**: $1 x 1 x 1$
* **Feature maps**: N/A
* **Kernel size**: N/A
* **Stride**: N/A
* **Padding**: N/A
* **Output size**: $(1x1)$
* **Neurons**: $84 * (1x1x1) = 84$
* **Trainable parameters**: $84*(120x1x1 + 1) = 10164$
* **Total connections**: $120 * (16x5x5x1) = 48000$
* **Activation function**: Tanh

The second-to-last layer in the LeNet-5 network is fully-connected to the convolutional layer $C5$; we can consider $F6$ to be a true fully-connected layer with $84$ units. Here the number of units for this layer was selected according to a $7x12$ bitmap representation corresponding to one of the $10$ character classes. This is considering that the representation used is not particularly useful for _isolated_ digits but rather _strings_ of characters selected from the printable ASCII set which tends to produce _confusable_ i.e., different characters with similar appearances (e.g., an upper/lower-case "O" and a zero, or an uppercase "I" and the digit 1).

##### Output layer (OUTPUT or F7)

* **Classes**: 10
* **Output size**: $(10x1)$
* **Neurons**: $84 * (1x1x1) = 84$
* **Trainable parameters**: $84*(10x1x1 + 0) = 840$
* **Total connections**: $10 * (84x1x1x1) = 840$
* **Activation function**: RBF

The final layer in the LeNet-5 is of course the classification output layer — a fully-connected layer composed of Euclidean Radial Basis Function units (RBF). There are 10 RBF units, one for each digit class, each with a set of 84 inputs. The output of each RBF unit is computed with a Euclidean distance summation between its input vector and its parameter vector. The Euclidean distance (L2 loss) serves as a penalty for the input and RBF digit class; the larger the RBF output, the farther away the input is from the parameter vector. The choice of L2 loss here greatly penalises outliers (distance "away" from parameter vectors). The RBF output can also be interpreted as the un-normalised negative log-likelihood of the Gaussian distribution of $F6$.

In short, the parameter vectors of the RBF units act as target vectors for the $F6$ layer. The components of the parameter vectors are valued $+1$ or $-1$, which are points along the maximum curvature of the scaled _hyperbolic tangent_, i.e., [tanh](https://en.wikipedia.org/wiki/Hyperbolic_functions#Hyperbolic_tangent), or in other words — the re-scaled and shifted sigmoid activation function. Therefore, the RBF activation function is chosen for this output layer to enforce the maximally non-linear range of the tanh function applied to units in $F6$.

#### Reflecting on the LeNet-5

##### Activation functions

The tanh and sigmoid activation functions used in LeNet-5 have their downsides. As pointed out in the original paper by LeCun et al., 1998, the scaled tanh function presents issues with saturation at the maximum and minimum boundary points. This leads to [vanishing gradient problem](https://en.wikipedia.org/wiki/Vanishing_gradient_problem) as the derivatives of the function approach zero. Since we have a handful of hidden layers, the multiplication of these near-zero gradient values results in an exponential decrease in the gradient as it is propagated from the final to the initial layer.

ReLu or Leaky ReLu are more often used nowadays to address this. ReLU imposes a lower-bound for gradient values at $0$. This handles the vanishing gradient problem by eliminating saturation at very small gradient values — their values simply become zero, ultimately encourages sparsity in the learned representations [3]. Yet in some cases, ReLU may end up always returning the lower-bound ($0$) for any input. The [dying ReLU problem](https://www.educative.io/answers/what-is-the-dying-relu-problem) is the result of zero-valued gradient propagations and a high learning rate or a large negative bias could be at fault. To mitigate this, the [Leaky ReLU](https://www.tensorflow.org/api_docs/python/tf/keras/layers/LeakyReLU) is used which does not produce zero-valued gradients.

## Details

Using the Keras API, you have to create a small convolutional neural networks using less than 15 layers, containing at least one convolutional layer, one pooling layer and one dense (fully connected layer). You can find a list of the different layers available [here](https://www.tensorflow.org/api_docs/python/tf/keras/layers).

In [ ]:
### From Udacity's `training.py`

In [ ]:
def create_network():
    net = tf.keras.models.Sequential()
    # IMPLEMENT THIS FUNCTION
    return net

You should experiment with different designs (number of layers, types of pooling, filter sizes, number of fully connected layers, number of neurons).

You will need to feed the image directory to `training.py` (`GTSRB/Final_Training/Images/`) with `-d`, and can view the final metrics visualization in the Desktop.

In [ ]:
### From Udacity's `utils.py`

In [ ]:
def get_module_logger(mod_name):
    logger = logging.getLogger(mod_name)
    handler = logging.StreamHandler()
    formatter = logging.Formatter('%(asctime)s %(levelname)-8s %(message)s')
    handler.setFormatter(formatter)
    logger.addHandler(handler)
    logger.setLevel(logging.DEBUG)
    return logger

In [ ]:
### From Udacity's `training.py`

In [ ]:
logger = get_module_logger(__name__)
parser = argparse.ArgumentParser(description='Download and process tf files')
parser.add_argument('-d', '--imdir', required=True, type=str,
                    help='data directory')
parser.add_argument('-e', '--epochs', default=10, type=int,
                    help='Number of epochs')
args = parser.parse_args()    

logger.info(f'Training for {args.epochs} epochs using {args.imdir} data')

In [ ]:
### From Udacity's `utils.py`

In [ ]:
def process(image,label):
    """ small function to normalize input images """
    image = tf.cast(image/255. ,tf.float32)
    return image,label

In [ ]:
def get_datasets(imdir):
    """ extract GTSRB dataset from directory """
    train_dataset = image_dataset_from_directory(imdir, 
                                       image_size=(32, 32),
                                       batch_size=32,
                                       validation_split=0.2,
                                       subset='training',
                                       seed=123,
                                       label_mode='int')

    val_dataset = image_dataset_from_directory(imdir, 
                                        image_size=(32, 32),
                                        batch_size=32,
                                        validation_split=0.2,
                                        subset='validation',
                                        seed=123,
                                        label_mode='int')
    train_dataset = train_dataset.map(process)
    val_dataset = val_dataset.map(process)
    return train_dataset, val_dataset

In [ ]:
### From Udacity's `training.py`

In [ ]:
# get the datasets
train_dataset, val_dataset = get_datasets(args.imdir)

In [ ]:
model = create_network()

model.compile(optimizer='adam',
          loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
          metrics=['accuracy'])
history = model.fit(x=train_dataset, 
                    epochs=args.epochs, 
                    validation_data=val_dataset)

In [ ]:
### From Udacity's `utils.py`

In [ ]:
def display_metrics(history):
    """ plot loss and accuracy from keras history object """
    f, ax = plt.subplots(1, 2, figsize=(15, 5))
    ax[0].plot(history.history['loss'], linewidth=3)
    ax[0].plot(history.history['val_loss'], linewidth=3)
    ax[0].set_title('Loss', fontsize=16)
    ax[0].set_ylabel('Loss', fontsize=16)
    ax[0].set_xlabel('Epoch', fontsize=16)
    ax[0].legend(['train loss', 'val loss'], loc='upper right')
    ax[1].plot(history.history['accuracy'], linewidth=3)
    ax[1].plot(history.history['val_accuracy'], linewidth=3)
    ax[1].set_title('Accuracy', fontsize=16)
    ax[1].set_ylabel('Accuracy', fontsize=16)
    ax[1].set_xlabel('Epoch', fontsize=16)
    ax[1].legend(['train acc', 'val acc'], loc='upper left')
    plt.show()

In [ ]:
display_metrics(history)

## Tips

A good starting point for small networks is LeNet5. You will find many existing implementations online.

Don't forget the basic structure of a convnet: convolutional layer, activation and pooling.

You can use the [`summary`](https://www.tensorflow.org/api_docs/python/tf/keras/Model#summary) method of the Keras model API to print the description of your model.

## Credits

This assignment was prepared by Thomas Hossler et al., Winter 2021 (link [here](https://www.udacity.com/course/self-driving-car-engineer-nanodegree--nd0013)).


References

[1] Yu, F. Vladlen, K. Multi-Scale Context Aggregation by Dialated Convolutions. ArXiv. 2015. [doi:10.48550/ARXIV.1511.07122](https://arxiv.org/abs/1511.07122).

[2] LeCun, Y. et al. "Gradient-based learning applied to document recognition." Proceedings of the IEEE, 86(11):2278-2324, 1998. [doi:10.1109/5.726791](https://ieeexplore.ieee.org/document/726791).


[3] Glorot, X. et al. Deep Sparse Rectifier Neural Networks. Proceedings of the Fourteenth International Conference on Artificial Intelligence and Statistics, PMLR 15:315-323, 2011. [https://proceedings.mlr.press/v15/glorot11a.html](https://proceedings.mlr.press/v15/glorot11a/glorot11a.pdf).


Helpful resources:
* [Convolutional Neural Network: An Overview by S. Shah | Analytics Vidhya](https://www.analyticsvidhya.com/blog/2022/01/convolutional-neural-network-an-overview/)
* [Understanding the receptive field of deep convolutional networks by N. Adaloglou | AI Summer](https://theaisummer.com/receptive-field/)
* [The Dying ReLU Problem, Clearly Explained by K. Leung | Towards Data Science](https://towardsdatascience.com/the-dying-relu-problem-clearly-explained-42d0c54e0d24)
* [What is the dying ReLU problem? by S. Azhar | Educative.io](https://www.educative.io/answers/what-is-the-dying-relu-problem)